## Sistemas de Recomendação Sequenciais

Nesse projeto, iremos implementar o algoritmo SASRec, que é um algoritmo de
recomendação sequencial.

Primeiramente, o que é um algoritmo de recomendação sequencial? Um algoritmo de
recomendação sequencial é uma subcategorial dos sistemas de recomendação, que
são uma categoria de sistemas que tem como objetivo predizer o filtrar itens
(como filmes, livros, etc.) que um usuário alvo irá gostar. Os sistemas de
recomendação sequenciais se diferem ao tomar em consideração o tempo, ou seja,
a ordem em que os usuários interagem com os itens do sistema, tentando predizer
a partir disso a sua próxima interação.

Esses sistemas são bastante semelhantes aos sistemas de recomendação baseados
em sessão, tendo bastante intersectação entre eles. A diferença principal é que
os sistemas de recomendação sequencial se baseia na ordem em que os usuários
interagem os itens, já os sistemas de recomendação baseados em sessão se baseiam
nos grupos de itens que um usuário interage com durante suas sessões de uso.

Nesse projeto iremos usar um conjunto de dados de reprodução de músicas do 
last.fm (bem antigo, com reproduções até o final de 2009). Tomaremos uma 
abordagem mista, onde separaremos a sequência dos usuários em blocos menores de
reprodução contínua de músicas que chamaremos de sessões de uso, já que isso nos
permitirá expandir os dados em mais blocos de sequências menores, o que
acreditamos fazer mais sentido no contexto de predizer a próxima música, já que
assumimos que o passado recente é mais relevante para a próxima música que
históricos distantes e temos uma limitação de janela de quantos itens podemos
escolher para o passado recente.

In [ ]:
import os
import sys
import copy
import time
import random
import warnings
import contextlib
from datetime import timedelta, datetime

import joblib
import numpy as np
import polars as pl
import plotly.graph_objects as go

from tqdm.notebook import tqdm
from IPython.display import display
from ipywidgets.widgets import HBox

import torch
from torch.functional import F
from torch import nn, optim, cuda
from torch.utils.data import DataLoader

## Google Colab

Configurando persistência de dados caso esteja rodando dentro do ambiente do Google Colab.

In [2]:
ROOT_PATH = './'
DRIVE_PATH = 'Colab/RecSys-TP'

# When on Colab, use Google Drive as the root path to persist and load data
if 'google.colab' in sys.modules:
    from google.colab import drive, output
    output.enable_custom_widget_manager()

    drive.mount('/content/drive')
    ROOT_PATH = os.path.join('/content/drive/My Drive/', DRIVE_PATH)
    os.makedirs(ROOT_PATH, exist_ok=True)
    os.chdir(ROOT_PATH)

## Configurações

Detectando o dispositivo a ser utilizado para treinamento (CPU ou GPU), além de outras configurações de treinamento.

In [ ]:
RANDOM_SEED = 1984

BATCH_SIZE = 512
MAX_SEQUENCE_LENGTH = 50
DROPOUT_PROB = 0.4
HIDDEN_DIM = 64
NUM_BLOCKS = 2

TOTAL_EPOCHS = 500

BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-8
AMSGRAD = False
WEIGHT_DECAY = 0.01

WARMUP_RATIO = 0.05
LEARNING_RATE = 0.04
USE_SCHEDULER = True

# LEARNING_RATE = 0.004
# USE_SCHEDULER = False

EVAL_K = 10


PYTORCH_DEVICE = 'cpu'

# Use NVIDIA GPU if available
if cuda.is_available():
    PYTORCH_DEVICE = 'cuda'

# Use Apple Metal backend if available
if torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("Your device supports MPS but it is not installed. Checkout https://developer.apple.com/metal/pytorch/")
    else:
        PYTORCH_DEVICE = 'mps'


print (f"Using {PYTORCH_DEVICE} device for PyTorch")

Using cuda device for PyTorch


In [ ]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.mps.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Carregamento dos dados

In [5]:
# Carrega os dados do dataset (http://ocelma.net/MusicRecommendationDataset/lastfm-1K.html)
user_profiles = pl.read_csv("./data/lastfm-dataset-1K/userid-profile.tsv", separator="\t")
user_interactions = pl.read_csv("./data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv", separator="\t", has_header=False, quote_char=None)

# Renomeia as colunas
user_profiles.columns = ["user_id", "gender", "age", "country", "registered"]
user_interactions.columns = ["user_id", "timestamp", "artist_id", "artist_name", "track_id", "track_name"]

# Descarta linhas com valores nulos nas iterações
user_interactions = user_interactions.drop_nulls()

display(user_profiles.sample(10, seed=RANDOM_SEED))
display(user_interactions.sample(10, seed=RANDOM_SEED))

user_id,gender,age,country,registered
str,str,i64,str,str
"""user_000809""","""m""",null,"""Finland""","""Jun 8, 2005"""
"""user_000112""","""f""",30,"""Turkey""","""Mar 25, 2006"""
"""user_000086""","""f""",27,null,"""Sep 21, 2007"""
"""user_000403""","""f""",null,"""United States""","""May 17, 2006"""
"""user_000863""",null,null,"""United Kingdom""","""Oct 15, 2004"""
"""user_000720""","""f""",null,"""Norway""","""Jun 29, 2007"""
"""user_000985""","""f""",null,"""Australia""","""May 22, 2006"""
"""user_000487""","""m""",null,"""Netherlands""","""Mar 8, 2006"""
"""user_000049""",null,null,null,"""Jan 11, 2006"""


user_id,timestamp,artist_id,artist_name,track_id,track_name
str,str,str,str,str,str
"""user_000806""","""2008-08-09T19:28:14Z""","""fc61dd75-880b-44ba-9ba9-c7b643…","""Prefuse 73""","""60f3a1c9-e756-4a48-8a3e-c44140…","""Altoid Addiction (Interlude)"""
"""user_000108""","""2007-12-19T17:14:42Z""","""6ae51665-8261-4ae5-883f-189965…","""Filter""","""784f24f6-6fa7-44e7-81a5-a7b592…","""The Best Things"""
"""user_000079""","""2008-12-01T00:09:19Z""","""48896dee-a985-424d-9849-84802f…","""Johnny Mathis""","""e77a742f-eb2c-417c-9b48-45e404…","""Can'T Get Out Of This Mood"""
"""user_000407""","""2008-05-20T15:37:40Z""","""8bfac288-ccc5-448d-9573-c33ea2…","""Red Hot Chili Peppers""","""7a3e8796-a0b3-4999-b268-4e2d47…","""Otherside"""
"""user_000861""","""2008-02-12T22:25:17Z""","""31aa6f87-8d00-4ae9-a5cc-6d7eee…","""Alphaville""","""f11939cf-9ad1-45b8-b927-a89b00…","""Big In Japan"""
"""user_000728""","""2009-05-21T19:04:24Z""","""41c86965-305a-482d-bc1e-2daeca…","""Skankfunk""","""e48c9ed7-34ac-44aa-ab5a-3d792a…","""Melo-Pole"""
"""user_000990""","""2009-03-31T10:20:28Z""","""1bc69a93-8020-4e07-8b05-0b6331…","""The Cliks""","""f89ba590-0226-4c65-b5c5-4b69ee…","""Complicated"""
"""user_000174""","""2005-05-03T18:42:12Z""","""fc178247-53b6-4702-ad77-546cb0…","""The Exposures""","""b222a53c-3168-43e5-a40d-65e95a…","""Sake Rock"""
"""user_000412""","""2005-11-04T22:08:08Z""","""86e736b4-93e2-40ff-9e1c-fb7c63…","""Barenaked Ladies""","""05b34070-535a-4b92-aa9b-ecb97c…","""Call And Answer"""


## Processando os dados

In [6]:
# Cria um mapeamento dos IDs para números
item_ids = user_interactions['track_id'].unique().to_list()
user_ids = user_interactions['user_id'].unique().to_list()


item_id_index = {id: i + 1 for i, id in enumerate(item_ids)}
item_id_index_rev = {v: k for k, v in item_id_index.items()}

user_id_index = {id: i + 1 for i, id in enumerate(user_ids)}
user_id_index_rev = {v: k for k, v in user_id_index.items()}


# Aplica as transformações no dataframe
dataset = user_interactions.select(
    pl.col('user_id').replace_strict(user_id_index).alias('uid'),
    pl.col('track_id').replace_strict(item_id_index).alias('iid'),
    pl.col('timestamp').cast(pl.Datetime).alias('ts')
).sort('uid', 'ts')

max_iid = dataset['iid'].max()

display(dataset.head(10))

uid,iid,ts
i64,i64,datetime[μs]
1,597423,2006-03-17 14:08:20
1,110737,2006-03-17 14:13:36
1,699216,2006-03-17 14:21:22
1,261464,2006-03-17 14:24:38
1,403066,2006-03-17 14:27:18
1,660618,2006-03-17 14:29:43
1,498360,2006-03-17 14:32:44
1,90850,2006-03-17 14:37:24
1,349638,2006-03-17 14:42:10


### Separando as sessões

Devido a natureza do Last.FM, cada usuário possui um histórico de músicas ao 
longo de um extenso período de tempo. Para melhorar a qualidade e usabilidade do
modelo, separamemos as reproduções em sessões, definindo o fim de uma sessão 
como um período de tempo de 30 minutos onde não houve nenhuma reprodução de 
música.

Desse modo conseguimos ter sessões de tamanho arbitrário que representam a 
reprodução contínua de músicas.

In [ ]:
threshold = timedelta(minutes=30)

# Marca cada música como se ela representa ou não o início de uma nova sessão
new_session_col = dataset.group_by('uid') \
    .agg(
        # Separa por usuário (uma sessão é de um usuário)
        pl.col('iid'), 

        # Computa a diferença entre a música atual e a anterior, se essa 
        # diferença for nula ou maior igual ao nosso limite de tempo,
        # então é um início de sessão.
        (pl.col('ts').diff().fill_null(threshold) >= threshold).alias('start') , 
    # Expande as duas listas simultaneamente para que voltemos ao formato inicial
    ).explode('iid', 'start')['start'] 

# Agora, para criar um id para cada sessão, vamos usar a função cum_sum (isso 
# funciona pois nossos dados estão ordenados por usuário e timestamp, 
# respectivamente)
dataset_with_session = dataset.with_columns(new_session_col.cum_sum().alias('sid'))

display(dataset_with_session)

uid,iid,ts,sid
i64,i64,datetime[μs],u32
1,597423,2006-03-17 14:08:20,1
1,110737,2006-03-17 14:13:36,1
1,699216,2006-03-17 14:21:22,1
1,261464,2006-03-17 14:24:38,1
1,403066,2006-03-17 14:27:18,1
…,…,…,…
992,136460,2009-04-30 17:30:16,899897
992,438858,2009-04-30 17:32:23,899897
992,103181,2009-04-30 17:34:48,899897


In [ ]:
# Agrupa as reproduções por sessão em um array
dataset_grouped = dataset_with_session.group_by('sid').agg(pl.col('iid').alias('iids'))
display(dataset_grouped.head())

# Imprime a distribuição do tamanho das sessões
display(dataset_grouped['iids'].list.len().describe())


# Reduz o número de amostras para reduzir o tempo de treinamento (ao custo de
# uma redução na qualidade da base de dados) Seguiremos com 100k sessões das
# ~900k presentes na base.
dataset_grouped = dataset_grouped.sample(1e5, shuffle=True)

sid,iids
u32,list[i64]
117591,"[832385, 225508, … 239103]"
154938,"[472615, 199975, … 117821]"
609790,"[562867, 653424, … 307823]"
142353,"[611574, 239273, … 718995]"
422608,"[907537, 907537, … 687487]"


statistic,value
str,f64
"""count""",899898.0
"""null_count""",0.0
"""mean""",18.871339
"""std""",43.066011
"""min""",1.0
"""25%""",3.0
"""50%""",9.0
"""75%""",21.0
"""max""",5435.0


### Separação dos dados de treino, teste e validação

Iremos dividir o dataset em 3 conjuntos de treino, teste e validação. Desta
forma, conseguimos avaliar ao longo do treinamento a qualidade do modelo
por meio dos dados de validação evitando um enviesamento para a avaliação final
utilizando os dados de teste.

Temos duas estratégias possíveis para separação aqui: a primeira seria
utilizar uma separação por sessão, ou seja, dedicar parte das sessões para
cada um dos conjuntos. Outra alternativa é manter todas as sessões nos 3
conjuntos, modificando removendo as últimas duas reproduções para os dados
de treino, a última para os dados de validação e mantendo os dados completos
para os dados de teste.

Seguiremos com a segunda estratégia, mantendo todas as sessões nos 3 conjuntos,
já que parece ter sido a estratégia tomada no artigo original, entretanto 
acreditamos que isso pode afetar a qualidade da avaliação final pela
similaridade com os dados de treino durante a validação e teste.

In [9]:
def train_slice(data: list[int]) -> list[int]:
    if len(data) < 3:
        return data
    
    return data[:-2]

def validation_slice(data: list[int]) -> tuple[list[int], int] | None:
    if len(data) < 3:
        return None
    
    return (data[:-2], data[-2])

def test_slice(data: list[int]) -> tuple[list[int], int] | None:
    if len(data) < 3:
        return None
    
    return (data[:-1], data[-1])

train_data = [train_slice(data.to_numpy()) for data in dataset_grouped['iids']]
validation_data = [validation_slice(data.to_numpy()) for data in dataset_grouped['iids'] if len(data) > 2]
test_data = [test_slice(data.to_numpy()) for data in dataset_grouped['iids'] if len(data) > 2]

## O modelo "Self-Attentive Sequential Recommendation" (SASRec)

O modelo SASRec foi proposto em 2018 pelos pesquisadores
[Wang-Cheng Kang e Julian McAuley](https://arxiv.org/abs/1808.09781). Ele
consiste de um modelo para recomendação sequencial baseado na (recente no 
momento de lançamento) arquitetura de atenção 
([Attention is All You Need](https://arxiv.org/abs/1706.03762)), construindo
assim uma rede neural profunda com embeddings, camadas de atenção e camadas de
avanço pontual para o propósito de gerar recomendações baseadas em dados 
sequenciais. Esse modelo foi um dos pioneiros nessa estratégia, resultando em
avanços significativos na qualidade das recomendações. Um dos seus pontos
limitantes é a não inclusão de informações contextuais (como dados dos itens e
usuários) para a geração das recomendações.

![Diagrama simplificado da estrutura do modelo - retirado do paper original](./assets/sasrec-diagram.png)

Esse modelo foi capaz de superar os resultados de outros modelos proeminentes
durante seu lançamento, como o BPR, FPMC, TransRec e GRU4Rec.

### A implementação

Partimos de uma implementação já existente em PyTorch que se baseia, 
indiretamente, na implementação original dos autores em TensorFlow:
[versão em PyTorch por _seanswyi_](https://github.com/seanswyi/sasrec-pytorch).

Fizemos algumas adaptações em relação ao original, porém sem afetar a estrutura
da rede em si, apenas mudando o otmizado utilizado, batch size, parâmetros e
alguns pequenos ajustes no código para fica mais organizado, marginalmente mais
eficiente e ao ambiente de desenvolvimento.

In [10]:
# The following SASRec implementation is an adaptation from https://github.com/seanswyi/sasrec-pytorch

InputSequences = torch.Tensor
PositiveSamples = torch.Tensor
NegativeSamples = torch.Tensor

class PointWiseFFNN(nn.Module):
    def __init__(self, hidden_dim: int) -> None:
        super().__init__()

        self.W1 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.W2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x_1 = self.relu(self.W1(x))
        x_2 = self.W2(x_1)

        return x_2


class SelfAttnBlock(nn.Module):
    def __init__(
        self,
        max_seq_len: int,
        hidden_dim: int,
        dropout_p: float,
        device: str,
    ) -> None:
        super().__init__()

        self.max_seq_len = max_seq_len
        self.layer_norm = nn.LayerNorm(normalized_shape=hidden_dim)
        self.dropout = nn.Dropout(p=dropout_p)

        self.self_attn = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=1,
            dropout=dropout_p,
            batch_first=True,
        )
        self.ffnn = PointWiseFFNN(hidden_dim=hidden_dim)

    def dropout_layernorm(self, x: torch.Tensor) -> torch.Tensor:
        layer_norm_output = self.layer_norm(x)
        dropout_output = self.dropout(layer_norm_output)

        return dropout_output

    def forward(self, x: torch.Tensor, padding_mask: torch.Tensor) -> torch.Tensor:
        seq_len = x.shape[1]
        attention_mask = ~torch.tril(
            torch.ones(size=(seq_len, seq_len), dtype=torch.bool, device=x.device.type)
        )

        x_attn, _ = self.self_attn(
            key=self.layer_norm(x),
            query=x,
            value=x,
            attn_mask=attention_mask,
        )
        x_attn_output = x + self.dropout_layernorm(x_attn)

        x_ffnn = self.ffnn(x_attn_output)
        x_ffnn_output = x_attn_output + self.dropout_layernorm(x_ffnn)

        output = x_ffnn_output * padding_mask.unsqueeze(-1)
        return output


class EmbeddingLayer(nn.Module):
    def __init__(
        self,
        num_items: int,
        hidden_dim: int,
        max_seq_len: int,
    ) -> None:
        super().__init__()

        self.hidden_dim = hidden_dim
        self.item_emb_matrix = nn.Embedding(
            num_embeddings=num_items + 1,
            embedding_dim=hidden_dim,
        )
        self.positional_emb = nn.Embedding(
            num_embeddings=max_seq_len,
            embedding_dim=hidden_dim,
        )

    def forward(self, x):
        x = self.item_emb_matrix(x)
        x *= self.hidden_dim ** 0.5

        batch_size = x.shape[0]
        seq_len = x.shape[1]
        device = x.device.type

        positions = torch.tile(torch.arange(seq_len, device=device), dims=(batch_size, 1))

        positional_embs = self.positional_emb(positions)
        x += positional_embs

        return x


class SASRec(nn.Module):
    def __init__(
        self,
        num_items: int,
        num_blocks: int,
        hidden_dim: int,
        max_seq_len: int,
        dropout_p: float,
        device: str,
    ) -> None:
        super().__init__()

        self.device = device

        self.embedding_layer = EmbeddingLayer(
            num_items=num_items,
            hidden_dim=hidden_dim,
            max_seq_len=max_seq_len,
        )
        self_attn_blocks = [
            SelfAttnBlock(
                max_seq_len=max_seq_len,
                hidden_dim=hidden_dim,
                dropout_p=dropout_p,
                device=device,
            )
            for _ in range(num_blocks)
        ]
        self.self_attn_blocks = nn.Sequential(*self_attn_blocks)

        self.dropout = nn.Dropout(p=dropout_p)
        self.layer_norm = nn.LayerNorm(normalized_shape=hidden_dim)

    def get_padding_mask(self, seqs: torch.Tensor) -> torch.Tensor:
        is_padding = seqs == 0
        padding_mask = ~is_padding

        return padding_mask

    def forward(
        self,
        input_seqs: torch.Tensor,
        item_idxs: torch.Tensor = None,
        positive_seqs: torch.Tensor = None,
        negative_seqs: torch.Tensor = None,
    ) -> torch.Tensor:
        padding_mask = self.get_padding_mask(seqs=input_seqs)

        input_embs = self.dropout(self.embedding_layer(input_seqs))
        input_embs *= padding_mask.unsqueeze(-1)

        # For loop because nn.Sequential can't handle multiple inputs.
        attn_output = input_embs
        for block in self.self_attn_blocks:
            attn_output = block(x=attn_output, padding_mask=padding_mask)
        attn_output = self.layer_norm(attn_output)

        if item_idxs is not None:  # Inference.
            item_embs = self.embedding_layer.item_emb_matrix(item_idxs)
            logits = attn_output @ item_embs.transpose(2, 1)
            logits = logits[:, -1, :]
            outputs = (logits,)
        elif (positive_seqs is not None) and (negative_seqs is not None):  # Training.
            positive_embs = self.dropout(self.embedding_layer(positive_seqs))
            negative_embs = self.dropout(self.embedding_layer(negative_seqs))

            positive_logits = (attn_output * positive_embs).sum(dim=-1)
            negative_logits = (attn_output * negative_embs).sum(dim=-1)

            outputs = (positive_logits,)
            outputs += (negative_logits,)

        return outputs

### Sobre os dados de entrada

Conforme feito originalmente, seguimos a estratégia de geramento de amostras
negativas para as recomendações, isto é, para cada recomendação esperada, 
geramos aleatoriamente exemplos de recomendações negativas ("errada"), desse
modo alcançando resultados melhores e expandindo o conjunto de dados.

Além disso, devido a natureza do modelo, é imporante que as sequências tenham
tamanho fixo, por conta disso é feito um processamento de _padding_ ou 
truncamento da amostra para que tenha esse tamanho. Isso também influenciou o
nosso mapeamento anterior dos itens em identificadores numéricos, inciando a
sequência a partir do 1, já que usaremos o 0 como identificador de padding.

In [ ]:
# Barra de progresso para a execução paralela no joblib
# Obtido em: https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution/58936697#58936697
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()
    
# Gera as amostras negativas para um item positivo
def gen_samples(
    positive_sample: int,
    num_items: int,
    num_samples: int,
    seed: int,
) -> tuple[int, list[int]]:
    
    np.random.seed(seed)

    negative_samples = []
    while len(negative_samples) < num_samples:
        negative_sample = np.random.randint(1, num_items + 1)
        if negative_sample not in negative_samples and negative_sample != positive_sample:
            negative_samples.append(negative_sample)

    return positive_sample, negative_samples
    

# Pré-computação das amostras negativas para todas as amostras positivas, 
# computado concorrentemente com o joblib, já que o processo pode ser demorado.
def get_positive2negatives(num_items: int, num_samples: int = 100) -> dict[int, list[int]]:
    all_samples = np.arange(1, num_items + 1)

    with warnings.catch_warnings(action="ignore"):
        with tqdm_joblib(tqdm(desc="Samples: ", total=len(all_samples))) as pbar:
            res = joblib.Parallel(n_jobs=-1)(joblib.delayed(gen_samples)(n, num_items, num_samples, np.random.randint(2e9)) for n in all_samples)
            positive2negatives = {k: v for k, v in res}

    return positive2negatives


positive2negatives = get_positive2negatives(num_items=max_iid)


def pad_or_truncate_seq(
    sequence: list[int],
    max_seq_len: int,
) -> torch.Tensor:
    """Pads or truncates sequences depending on max_seq_len."""
    if isinstance(sequence, list) or isinstance(sequence, np.ndarray):
        sequence = torch.tensor(sequence)

    if len(sequence) > max_seq_len:
        sequence = sequence[-max_seq_len:]
    else:
        diff = max_seq_len - len(sequence)
        sequence = F.pad(sequence, pad=(diff, 0))

    return sequence

def collate_fn_train(
    batch: list[list[int]], 
    max_seq_len: int = MAX_SEQUENCE_LENGTH
) -> torch.Tensor:
    seq_tensors = [pad_or_truncate_seq(seq, max_seq_len=max_seq_len) for seq in batch]
    return torch.stack(seq_tensors)

def collate_fn_eval(
    batch: list[tuple[list[int], int]], 
    max_seq_len: int = MAX_SEQUENCE_LENGTH
) -> tuple[torch.Tensor, torch.Tensor]:
    seq_tensors = [pad_or_truncate_seq(seq[0], max_seq_len=max_seq_len) for seq in batch]
    input_seqs = torch.stack(seq_tensors)

    item_idxs = [x[1] for x in batch]
    item_idxs = torch.tensor(item_idxs, dtype=torch.long)

    return input_seqs, item_idxs

def build_train_dataloader(
    data: list[list[int]],
    batch_size: int = BATCH_SIZE,
    max_seq_len: int = MAX_SEQUENCE_LENGTH,
) -> DataLoader:
    return DataLoader(
        dataset=data,
        batch_size=batch_size,
        collate_fn=lambda batch: collate_fn_train(batch, max_seq_len=max_seq_len),
        shuffle=True,
    )

def build_eval_dataloader(
    data: list[tuple[list[int], int]],
    batch_size: int = BATCH_SIZE,
    max_seq_len: int = MAX_SEQUENCE_LENGTH,
) -> DataLoader:
    global positive2negatives

    input_seqs = [d[0] for d in data]
    all_pred_item_idxs = []

    for (_, positive_sample) in data:
        negative_samples = positive2negatives[positive_sample]
        all_pred_item_idxs.append([positive_sample] + negative_samples)
    
    # Join the inputs with the predictions (positive, ...negatives)
    final_data = list(zip(input_seqs, all_pred_item_idxs))

    return DataLoader(
        dataset=final_data,
        batch_size=batch_size,
        collate_fn=lambda batch: collate_fn_eval(batch, max_seq_len=max_seq_len),
        shuffle=False,
    )


Samples:   0%|          | 0/961416 [00:00<?, ?it/s]

In [ ]:
train_loader = build_train_dataloader(data=train_data, batch_size=BATCH_SIZE, max_seq_len=MAX_SEQUENCE_LENGTH)
test_loader = build_eval_dataloader(data=test_data, batch_size=BATCH_SIZE, max_seq_len=MAX_SEQUENCE_LENGTH)
validation_loader = build_eval_dataloader(data=validation_data, batch_size=BATCH_SIZE, max_seq_len=MAX_SEQUENCE_LENGTH)

## Treinamento

Por fim chegamos ao treinamento do modelo. Usaremos duas métricas de avaliação,
o NDCG@10 (Normalized Discounted Cumulative Gain nas 10 primeiras recomendações)
e o HIT@10 (Hit Rate nas 10 primeiras recomendações). Para mais informações sobre,
recomendamos a leitura na página do [EvidentlyAI](https://www.evidentlyai.com/ranking-metrics/evaluating-recommender-systems#hit-rate).
Que explica melhor essas métricas.

Para o treinamento, usamos um _batch size_ de 512 e um _scheduler_ para a taxa
de aprendizado (OneCycleLR), com um warmup de 5% e um _learning rate_ máximo de 
0.04. Para o otimizador usamos o AdamW e como função de perda usamos o 
BCELossWithLogits (Binary Cross Entropy with Logits).

Mais informações sobre os parâmetros de treinamento estão no começo do notebook.

In [13]:
# TODO: maybe pre-compute this?
def get_negative_samples(
    positive2negatives: dict[int, list[int]],
    positive_seqs: torch.Tensor,
    num_samples=1,
) -> torch.Tensor:
    negative_seqs = torch.zeros(size=positive_seqs.shape, dtype=torch.long)
    for row_idx in range(positive_seqs.shape[0]):
        for col_idx in range(positive_seqs[row_idx].shape[0]):
            positive_sample = positive_seqs[row_idx][col_idx].item()

            if positive_sample == 0:
                continue

            negative_samples = positive2negatives[positive_sample]
            negative_sample = np.random.choice(
                a=negative_samples, size=(num_samples,), replace=False
            )
            negative_seqs[row_idx][col_idx] = negative_sample[0]

    return negative_seqs

def compute_loss(
    positive_idxs: torch.Tensor,
    negative_idxs: torch.Tensor,
    positive_logits: torch.Tensor,
    negative_logits: torch.Tensor,
) -> torch.Tensor:
    global bce_criterion, PYTORCH_DEVICE

    positive_logits = positive_logits[positive_idxs]
    positive_labels = torch.ones(size=positive_logits.shape, device=PYTORCH_DEVICE)

    negative_logits = negative_logits[negative_idxs]
    negative_labels = torch.zeros(size=negative_logits.shape, device=PYTORCH_DEVICE)

    positive_loss = bce_criterion(positive_logits, positive_labels)
    negative_loss = bce_criterion(negative_logits, negative_labels)

    return positive_loss + negative_loss

def evaluate_model(
    model: SASRec,
    loader: DataLoader,
    device: str = PYTORCH_DEVICE,
    autocast: bool = False,
    autocast_dtype: torch.dtype = torch.bfloat16,
) -> tuple[float, float]:
    global EVAL_K
    ndcg = 0
    hit = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating Validation: ", total=len(loader), leave=False):
            input_seqs, item_idxs = batch
            total += input_seqs.shape[0]

            input_seqs = input_seqs.to(device)
            item_idxs = item_idxs.to(device)

            if autocast:
                with torch.amp.autocast(device_type=device, dtype=autocast_dtype):
                    outputs = model(input_seqs, item_idxs=item_idxs)
            else:
                outputs = model(input_seqs, item_idxs=item_idxs)

            logits = -outputs[0]

            # Metal shenanigans
            if logits.device.type == 'mps':
                logits = logits.detach().cpu()
            
            ranks = logits.argsort().argsort()
            ranks = [r[0].item() for r in ranks]

            for rank in ranks:
                if rank < EVAL_K:
                    ndcg += 1 / np.log2(rank + 2)
                    hit += 1
        
    ndcg /= total
    hit /= total

    return ndcg, hit

def train_model(
    model: SASRec,
    optimizer: optim.Optimizer,
    scheduler: optim.lr_scheduler.LRScheduler | None,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_epochs: int,
    device: str = PYTORCH_DEVICE,
    autocast: bool = False,
    autocast_dtype: torch.dtype = torch.bfloat16,
) -> tuple[list[float], list[float], list[float], tuple[dict, dict, dict], tuple[dict, dict, dict]]:
    global positive2negatives

    best_ndcg = 0
    best_hit = 0
    best_ndcg_epoch = 0
    best_hit_epoch = 0

    losses = []
    ndcgs = []
    hits = []
    lrs = []

    best_ncdg_model_state = None
    best_ncdg_optimizer_state = None
    best_ncdg_scheduler_state = None

    best_hit_model_state = None
    best_hit_optimizer_state = None 
    best_hit_scheduler_state = None

    # Plot the loss and other metrics

    fig_loss_widget = go.FigureWidget(layout=go.Layout(title="Loss"))
    fig_ndcg_widget = go.FigureWidget(layout=go.Layout(title="NDCG@" + str(EVAL_K)))
    fig_hit_widget = go.FigureWidget(layout=go.Layout(title="HIT@" + str(EVAL_K)))
    fig_lr_widget = go.FigureWidget(layout=go.Layout(title="Learning Rate"))


    fig_loss_widget.add_scatter(x=np.arange(len(losses)) + 1, y=losses)
    fig_ndcg_widget.add_scatter(x=np.arange(len(ndcgs)) + 1, y=ndcgs)
    fig_hit_widget.add_scatter(x=np.arange(len(hits)) + 1, y=hits)
    fig_lr_widget.add_scatter(x=np.arange(len(lrs)) + 1, y=lrs)

    fig_loss_widget.update_xaxes(title_text='Epoch')
    fig_ndcg_widget.update_xaxes(title_text='Epoch')
    fig_hit_widget.update_xaxes(title_text='Epoch')
    fig_lr_widget.update_xaxes(title_text='Epoch')
    
    fig_loss_widget.update_yaxes(title_text='Loss', type='log')
    fig_ndcg_widget.update_yaxes(title_text='NDCG@' + str(EVAL_K))
    fig_hit_widget.update_yaxes(title_text='Epoch@' + str(EVAL_K))
    fig_lr_widget.update_yaxes(title_text='Learning Rate')

    display(HBox([fig_loss_widget, fig_lr_widget]))
    display(HBox([fig_ndcg_widget, fig_hit_widget]))

    # Wait for widgets to load
    time.sleep(1)

    steps = 0
    for epoch in tqdm(range(num_epochs), desc="Epoch: "):
        model.train()
        epoch_loss = 0

        lrs.append([pg['lr'] for pg in optimizer.param_groups][0])

        for batch in tqdm(train_loader, desc="Training: ", total=len(train_loader), leave=False):
            model.zero_grad()

            positive_seqs = batch.clone()
            positive_idxs = torch.where(positive_seqs != 0)

            batch[:, -1] = 0
            input_seqs = batch.roll(shifts=1)
            negative_seqs = get_negative_samples(positive2negatives, positive_seqs)
            negative_idxs = torch.where(negative_seqs != 0)

            input_seqs = input_seqs.to(device)
            positive_seqs = positive_seqs.to(device)
            negative_seqs = negative_seqs.to(device)

            if autocast:
                with torch.amp.autocast(device_type=device, dtype=autocast_dtype):
                    output = model(input_seqs, positive_seqs=positive_seqs, negative_seqs=negative_seqs)
        
                    positive_logits = output[0]
                    negative_logits = output[1]
        
                    loss = compute_loss(positive_idxs, negative_idxs, positive_logits, negative_logits)
            else:
                output = model(input_seqs, positive_seqs=positive_seqs, negative_seqs=negative_seqs)
    
                positive_logits = output[0]
                negative_logits = output[1]
    
                loss = compute_loss(positive_idxs, negative_idxs, positive_logits, negative_logits)
                
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()

            if scheduler is not None:
                scheduler.step()
            
            steps += 1

        ndcg, hit = evaluate_model(model, val_loader, device=device, autocast=autocast, autocast_dtype=autocast_dtype)

        if ndcg > best_ndcg:
            best_ndcg = ndcg
            best_ndcg_epoch = epoch
            best_ncdg_model_state = copy.deepcopy(model.state_dict())
            best_ncdg_optimizer_state = copy.deepcopy(optimizer.state_dict())
            if scheduler is not None:
                best_ncdg_scheduler_state = copy.deepcopy(scheduler.state_dict())
        
        if hit > best_hit:
            best_hit = hit
            best_hit_epoch = epoch
            best_hit_model_state = copy.deepcopy(model.state_dict())
            best_hit_optimizer_state = copy.deepcopy(optimizer.state_dict())
            if scheduler is not None:
                best_hit_scheduler_state = copy.deepcopy(scheduler.state_dict())
        
        losses.append(epoch_loss)
        ndcgs.append(ndcg)
        hits.append(hit)

        
        fig_loss_widget.data[0].x = np.arange(len(losses)) + 1
        fig_loss_widget.data[0].y = losses
        fig_ndcg_widget.data[0].x = np.arange(len(ndcgs)) + 1
        fig_ndcg_widget.data[0].y = ndcgs
        fig_hit_widget.data[0].x = np.arange(len(hits)) + 1
        fig_hit_widget.data[0].y = hits
        fig_lr_widget.data[0].x = np.arange(len(lrs)) + 1
        fig_lr_widget.data[0].y = lrs

    print(f"Best NDCG@{EVAL_K} Epoch: {best_ndcg_epoch + 1}, NDCG@{EVAL_K}: {best_ndcg:.4f}")
    print(f"Best HIT@{EVAL_K} Epoch: {best_hit_epoch + 1}, HIT@{EVAL_K}: {best_hit:.4f}")
    
    return losses, ndcgs, hits, \
        (best_ncdg_model_state, best_ncdg_optimizer_state, best_ncdg_scheduler_state), \
        (best_hit_model_state, best_hit_optimizer_state, best_hit_scheduler_state)

In [ ]:
model = SASRec(num_items=max_iid, num_blocks=NUM_BLOCKS, hidden_dim=HIDDEN_DIM, 
               max_seq_len=MAX_SEQUENCE_LENGTH, dropout_p=DROPOUT_PROB, device=PYTORCH_DEVICE)

model.to(PYTORCH_DEVICE)

bce_criterion = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2), eps=EPS, weight_decay=WEIGHT_DECAY, amsgrad=AMSGRAD)

scheduler = None
if USE_SCHEDULER:
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=LEARNING_RATE,
        total_steps=TOTAL_EPOCHS * len(train_loader),
        pct_start=WARMUP_RATIO,
        anneal_strategy="linear",
    )

results = train_model(model, optimizer, scheduler, train_loader, validation_loader, TOTAL_EPOCHS, device=PYTORCH_DEVICE, autocast=False)

In [ ]:
losses, ndcgs, hits, \
    (best_ncdg_model_state, best_ncdg_optimizer_state, best_ncdg_scheduler_state), \
    (best_hit_model_state, best_hit_optimizer_state, best_hit_scheduler_state) = results

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(f"models/sasrec/{timestamp}/", exist_ok=True)

torch.save(best_ncdg_model_state, f"models/sasrec/{timestamp}/best_ncdg_model_state.pt")
torch.save(best_hit_model_state, f"models/sasrec/{timestamp}/best_hit_model_state.pt")

torch.save(best_ncdg_optimizer_state, f"models/sasrec/{timestamp}/best_ncdg_optimizer_state.pt")
torch.save(best_hit_optimizer_state, f"models/sasrec/{timestamp}/best_hit_optimizer_state.pt")

torch.save(best_ncdg_scheduler_state, f"models/sasrec/{timestamp}/best_ncdg_scheduler_state.pt")
torch.save(best_hit_scheduler_state, f"models/sasrec/{timestamp}/best_hit_scheduler_state.pt")

## Resultados

Após 3 horas e 50 minutos de treinamento (em 1x RTX 3080TI) foi realizado o 
treino de 500 epochs com a amostra de 100k sessões. Analisando os gráficos
de treinamento, conseguimos ver que o modelo não convergiu completamente, tendo
ainda potencial para melhora caso fosse treinado por mais tempo.

Os resultados obtidos foram:

| Métrica | Valor |
| --- | --- |
| NDCG@10 Validação | 0.681 |
| HIT@10 Validação | 0.777 |
| NDCG@10 Teste | 0.653 |
| HIT@10 Teste | 0.749 |

In [16]:
fig = go.Figure(layout = go.Layout(title="Loss"))
fig.add_scatter(x=np.arange(len(losses)) + 1, y=losses)
fig.update_xaxes(title_text='Epoch', type='log')
fig.update_yaxes(title_text='Loss')
display(fig)

fig = go.Figure(layout = go.Layout(title="NDCG@" + str(EVAL_K)))
fig.add_scatter(x=np.arange(len(ndcgs)) + 1, y=ndcgs)
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='NDCG@' + str(EVAL_K))
display(fig)

fig = go.Figure(layout = go.Layout(title="HIT@" + str(EVAL_K)))
fig.add_scatter(x=np.arange(len(hits)) + 1, y=hits)
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Epoch@' + str(EVAL_K))
display(fig)

In [17]:
# Load the best model during training
test_model = SASRec(num_items=max_iid, num_blocks=NUM_BLOCKS, hidden_dim=HIDDEN_DIM,
                    max_seq_len=MAX_SEQUENCE_LENGTH, dropout_p=DROPOUT_PROB, device=PYTORCH_DEVICE)
test_model.to(PYTORCH_DEVICE)
test_model.load_state_dict(best_ncdg_model_state)

ndcg, hit = evaluate_model(test_model, test_loader, device=PYTORCH_DEVICE, autocast=False)

print(f"Test NDCG@{EVAL_K}: {ndcg:.4f}")
print(f"Test HIT@{EVAL_K}: {hit:.4f}")

Evaluating Validation:   0%|          | 0/157 [00:00<?, ?it/s]

Test NDCG@10: 0.6531
Test HIT@10: 0.7493


## Referências

- [Self-Attentive Sequential Recommendation](https://cseweb.ucsd.edu/~jmcauley/pdfs/icdm18.pdf)
- [SASRec em PyTorch (por Pmixer)](https://github.com/pmixer/SASRec.pytorch)
- [SASRec em PyTorch (por Seanswyi)](https://github.com/seanswyi/sasrec-pytorch)
- [Métricas: NDCG e HIT](https://www.evidentlyai.com/ranking-metrics/evaluating-recommender-systems#hit-rate)
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)